In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [52]:
data = pd.read_csv('train.csv')

In [53]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

In [55]:
print (m, n)

42000 785


In [56]:
train_data = data[:int(0.8*m), :]
val_data = data[int(0.8*m):m, :]

X_train = train_data[:, 1:].T
X_train = X_train / 255.0
Y_train = train_data[:,0]

X_val = val_data[:,1:].T
X_val = X_val / 255.0
Y_val = val_data[: ,0]


In [57]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

(784, 33600)
(33600,)
(784, 8400)
(8400,)


In [58]:
def initialize_parameters():
    W1 = np.random.rand(10, 784) - 0.5
    B1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    B2 = np.random.rand(10, 1) - 0.5
    return W1, B1, W2, B2

def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    return np.exp(Z) / sum(np.exp(Z))

def forward_propagation(X, W1, B1, W2, B2):
    Z1 = np.dot(W1, X) + B1
    A1 = ReLU(Z1)
    Z2 = np.dot(W2, A1) + B2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def one_hot_Y(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    return one_hot_Y.T

def backward_propagation(X, Y, W1, B1, W2, B2, Z1, A1, Z2, A2):
    Y = one_hot_Y(Y)
    dZ2 = A2 - Y
    dW2 = np.dot(dZ2, A1.T) / m
    dB2 = np.sum(dZ2) / m
    dZ1 = W2.T.dot(dZ2)*(Z1 > 0)
    dW1 = np.dot(dZ1, X.T) / m
    dB1 = np.sum(dZ1) / m
    return dW1, dB1, dW2, dB2

def update_parameters(W1, B1, W2, B2, dW1, dB1, dW2, dB2, learning_rate):
    W1 -= learning_rate * dW1
    B1 -= learning_rate * dB1
    W2 -= learning_rate * dW2
    B2 -= learning_rate * dB2
    return W1, B1, W2, B2

def get_predictions(A2):
    return np.argmax(A2, axis=0)

def get_accuracy(predictions, Y):
    return np.mean(predictions == Y)

def gradient_descent(X, Y, lr, iterations):
    W1, B1, W2, B2 = initialize_parameters()

    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_propagation(X, W1, B1, W2, B2)
        dW1, dB1, dW2, dB2 = backward_propagation(X, Y, W1, B1, W2, B2, Z1, A1, Z2, A2)
        W1, B1, W2, B2 = update_parameters(W1, B1, W2, B2, dW1, dB1, dW2, dB2, lr)

        if i % 100 == 0:
            predictions = get_predictions(A2)
            accuracy = get_accuracy(predictions, Y)
            print(f"Iteration {i}, Accuracy: {accuracy:.4f}")
    return W1, B1, W2, B2

In [59]:
W1, B1, W2, B2 = gradient_descent(X_train, Y_train, 0.01, 10001)

Iteration 0, Accuracy: 0.1358
Iteration 100, Accuracy: 0.1669
Iteration 200, Accuracy: 0.2366
Iteration 300, Accuracy: 0.3036
Iteration 400, Accuracy: 0.3555
Iteration 500, Accuracy: 0.3960
Iteration 600, Accuracy: 0.4322
Iteration 700, Accuracy: 0.4659
Iteration 800, Accuracy: 0.5012
Iteration 900, Accuracy: 0.5369
Iteration 1000, Accuracy: 0.5686
Iteration 1100, Accuracy: 0.5951
Iteration 1200, Accuracy: 0.6196
Iteration 1300, Accuracy: 0.6423
Iteration 1400, Accuracy: 0.6619
Iteration 1500, Accuracy: 0.6782
Iteration 1600, Accuracy: 0.6940
Iteration 1700, Accuracy: 0.7086
Iteration 1800, Accuracy: 0.7205
Iteration 1900, Accuracy: 0.7305
Iteration 2000, Accuracy: 0.7389
Iteration 2100, Accuracy: 0.7468
Iteration 2200, Accuracy: 0.7546
Iteration 2300, Accuracy: 0.7618
Iteration 2400, Accuracy: 0.7674
Iteration 2500, Accuracy: 0.7735
Iteration 2600, Accuracy: 0.7782
Iteration 2700, Accuracy: 0.7820
Iteration 2800, Accuracy: 0.7859
Iteration 2900, Accuracy: 0.7894
Iteration 3000, Accura

Validation

In [60]:
def compute_validation_accuracy(X_val, Y_val, W1, B1, W2, B2):
    _, _, _, A2_val = forward_propagation(X_val, W1, B1, W2, B2)
    val_predictions = get_predictions(A2_val)
    val_accuracy = get_accuracy(val_predictions, Y_val)
    return val_accuracy


In [61]:
val_accuracy = compute_validation_accuracy(X_val, Y_val, W1, B1, W2, B2)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")


Validation Accuracy: 86.76%


Test accuracy 

In [62]:
test_data = pd.read_csv("test.csv")
test_data = np.array(test_data)
X_test = test_data.T / 255.0


If the test file has labels

In [63]:
Y_test = test_data[:, 0]
X_test = test_data[:, 1:].T / 255.0


In [64]:
def compute_test_accuracy(X_test, Y_test, W1, B1, W2, B2):
    _, _, _, A2_test = forward_propagation(X_test, W1, B1, W2, B2)
    test_predictions = get_predictions(A2_test)
    test_accuracy = get_accuracy(test_predictions, Y_test)
    return test_accuracy


In [65]:
test_accuracy = compute_test_accuracy(X_test, Y_test, W1, B1, W2, B2)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Test Accuracy: 87.64%
